# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


In [32]:
import xlwings as xw
import numpy as np


In [26]:
xw.__version__

'0.24.3'

In [28]:
#!pip install xlwings==0.24.3

## Задачи для совместного разбора

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

In [43]:
wb = xw.Book('07_excel_data_себестоимостьА_в1 (1).xlsx')

In [44]:
sheet = wb.sheets['Рецептура']
sheet.range('G7:O10').value

[[0.2, 0.4, 0.02, 0.02, 0.01, 0.005, 0.1, None, None],
 [0.07, 0.25, 0.01, 0.06, 0.015, None, 0.06, 0.05, None],
 [0.12, 0.22, 0.005, None, 0.009, 0.005, None, 0.05, 0.03],
 [0.12, 0.23, 0.01, None, 0.015, None, None, None, 0.05]]

In [45]:
consumption = sheet.range('G7:O10').options(np.array).value
consumption 

array([[0.2  , 0.4  , 0.02 , 0.02 , 0.01 , 0.005, 0.1  ,   nan,   nan],
       [0.07 , 0.25 , 0.01 , 0.06 , 0.015,   nan, 0.06 , 0.05 ,   nan],
       [0.12 , 0.22 , 0.005,   nan, 0.009, 0.005,   nan, 0.05 , 0.03 ],
       [0.12 , 0.23 , 0.01 ,   nan, 0.015,   nan,   nan,   nan, 0.05 ]])

In [46]:
unit_price = sheet.range('G14:O14').options(np.array).value
unit_price 

array([  7.,  10., 184.,  19.,   7., 290., 105.,  83., 260.])

In [47]:
cost = (np.nan_to_num(consumption) * unit_price).sum(axis=1)

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

In [49]:
sheet.range('T7:T10').options(transpose=True).value = cost
sheet.range('T6').value = 'Себестоимость'

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

In [ ]:
sheet.range('T4:T6').api.merge()

4. Выполнить 3 с помощью "протягиваемых" формул.

In [ ]:
from xlwings.constants import AutoFillType

In [ ]:
sheet.range('V7').formula = '=SUMPRODUCT(G7:O7, $G$14:$O$14)'
sheet.range('V7').api.autofill(destination=sheet.range('V7:V10').api,
                               type=AutoFillType.xlFillDefault)

In [ ]:
import pandas as pd

## Лабораторная работа 7.1

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [ ]:
recipes = pd.read_csv('recipes_sample.csv', sep=',',
                     usecols=['id', 'name', 'minutes', 'submitted', 'description', 'n_ingredients'])
reviews = pd.read_csv('reviews_sample.csv', sep=',', index_col=0)

2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [ ]:
import xlwings as xw
from xlwings.constants import AutoFillType
from xlwings import constants 
import numpy as np

In [ ]:
recipes_sample = recipes.sample(frac=0.05, random_state=1)
reviews_sample = reviews.sample(frac=0.05, random_state=1)

In [ ]:
wb = xw.Book('./result/recipes.xlsx')

wb.sheets['Sheet1'].name = 'Отзывы'
wb.sheets.add('Рецепты')

recipes_sheet = wb.sheets['Рецепты']
reviews_sheet = wb.sheets['Отзывы']

recipes_sheet.range('A1').options(pd.DataFrame, header=1, index=False, expand='table').value = recipes_sample
reviews_sheet.range('A1').options(pd.DataFrame, header=1, index=False, expand='table').value = reviews_sample

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [ ]:
arr = np.array(recipes_sample.minutes * 60)

recipes_sheet.range('g1').value = 'seconds_assign'
recipes_sheet.range('g2:g1501').options(transpose=True).value = arr

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [ ]:
ecipes_sheet.range('h1').value = 'seconds_formula'
recipes_sheet.range('h2').formula = '=D2 * 60'
recipes_sheet.range('h2').api.autofill(destination=recipes_sheet.range('h2:h1501').api,
                               type=AutoFillType.xlFillDefault)

5. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [ ]:
recipes_sheet.range('i1').value = 'n_reviews'
recipes_sheet.range('i2').value = "=COUNTIF(Отзывы!$B$2:$B$6336;Рецепты!B2)"
recipes_sheet.range('i2').api.autofill(destination=recipes_sheet.range('i2:i1501').api,
                               type=AutoFillType.xlFillDefault)

6. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [ ]:
recipes_sheet.range('g1:i1').api.font_object.font_style.set('bold')

recipes_sheet.range('g1:i1').api.font_object.horizontalalignment = xw.constants.Constants.xlCenter

7. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [ ]:
arr7 = recipes_sheet.range('C2:C1501').options(np.array)

for cell in arr7:
    if cell.value < 5:
        cell.color = (0, 255, 0)
    elif cell.value in range(5,10):
        cell.color = (255, 255, 0)
    else:
        cell.color = (255, 0, 0)

## Лабораторная работа 7.2

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

In [41]:
def validate(arr1, arr2):
    lst_res = []
    rows = reviews_sheet.range('A2:E6336').current_region.last_cell.row
    for row in range(rows-1):
        if (not arr1[row,3].value in range(0,5)):
            arr1[row,:].color = (255, 0, 0)
        elif not arr1[row,1].value in arr2.value:
            lst_res.append(arr1[row,1].value)
            arr1[row,:].color = (255, 0, 0)
    return lst_res

In [ ]:
arr8 = reviews_sheet.range('A2:E6336').options(np.array)
arr8_1 = recipes_sheet.range('B2:B1501').options(np.array)
len(validate(arr8, arr8_1)) #кол-во соответвующих строк

9. В файле `recipes_model.csv` находится модель данных предметной области "рецепты". При помощи пакета `csv` считайте эти данные. При помощи пакета `xlwings` запишите данные на лист `Модель` книги `recipes_model.xlsx`, начиная с ячейки `A2`, не используя циклы. Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука. 

10. При помощи пакета `xlwings` добавьте в столбец J формулу для описания столбца на языке SQL. Формула должна реализовывать следующую логику:

    1\. в начале строки идут значения из столбцов В и C (значение столбца С приведено к верхнему регистру), разделенные пробелом
    
    2\. далее идут слова на основе столбца "Ключ"
        2.1 если в столбце "Ключ" указано значение "PK", то дальше через пробел идет ключевое слово "PRIMARY KEY"
        2.2 если в столбце "Ключ" указано значение "FK", то дальше через пробел идет ключевое слово "REFERENCES", затем значения столбцов H и I в формате "название_таблицы(название_столбца)"
        
    3\. если в столбце "Обязательно к заполнению" указано значение "Y" и в столбце "Ключ" указано не "PK", то дальше через пробел идет ключевое слово "NOT NULL".

Заполните этой формулой необходимое количество строк, используя "протягивание". Количество строк для протягивания определите на основе данных.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

11. При помощи пакета `xlwings` измените стилизацию листа `Модель`.
* для заголовков добавьте заливку цвета `00ccff`
* примените автоподбор ширины столбца;
* сделайте шрифт заголовков полужирным;
* добавьте таблице автофильтр.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

12. Посчитайте количество атрибутов для каждой из сущностей. Создайте лист `Статистика` и запишите в него результат группировки, начиная с ячейки "А1". Визуализируйте полученный результат при помощи столбчатой диаграммы. Сохраните полученную визуализацию на лист `Статистика`, начиная с ячейки "E2".  Сделайте скриншот листа `Статистика` и прикрепите в ячейку ноутбука.

* Вы можете воспользоваться методами для визуализации, которые поставляются вместе с объектами `pandas` (см. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot) 